In [1]:
#installing all the required libraries
#pip install requests beautifulsoup4 pandas lxml selenium nltk vaderSentiment wordcloud matplotlib seaborn undetected-chromedriver

1. DELL REVIEW

In [2]:
#WEBSCRAPPING CODE FOR DELL 
#THIS WILL ONLY WORK FOR DELL WEBSITE
#IN WEBSCRAPPING WE HAVE TO MATCH THE WEBSITE PARAMETERS
#FOR THAT WE SHOULD USE INSPECT OPTION IN THE CORRESPONDING WEBPAGE SECTION AND MODIFY THE BASIC TEMPLATE AS PER THAT from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [3]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [4]:
driver = webdriver.Chrome(options=options)
url = "https://www.dell.com/en-us/shop/cty/pdp/spd/alienware-area-51-aa18250-gaming-laptop/useaa18250wcto15#ratings_section"
driver.get(url)
time.sleep(5)

In [5]:
try:
    reviews_section = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "ratings_section"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", reviews_section)
    time.sleep(3)
except Exception as e:
    print(f"Could not find reviews section: {e}")

In [6]:
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".pr-review"))
)

<selenium.webdriver.remote.webelement.WebElement (session="590da41fc01a058dbc91bc9b64af5f76", element="f.0AA7C5F5956965CC76F1EC997B3927CA.d.86373FDB341724F9E2913B7487872F75.e.227")>

In [7]:
def extract_review():
    page=[]
    review_elements=driver.find_elements(By.CSS_SELECTOR, ".pr-review")
    for rev in review_elements:
        try:
            rating_el=rev.find_element(By.CSS_SELECTOR, ".pr-snippet-stars")
            rating_label=rating_el.get_attribute("aria-label")
            rating=rating_label.split()[1] if rating_label else "N/A"
            try:
                headline=rev.find_element(By.CSS_SELECTOR, ".pr-rd-review-headline").text
            except:
                headline=""
            text=rev.find_element(By.CSS_SELECTOR, ".pr-rd-description-text").text
            try:
                author=rev.find_element(By.CSS_SELECTOR, ".pr-rd-author-nickname span:last-child").text
            except:
                author="Anonymous"
            try:
                date = rev.find_element(By.CSS_SELECTOR, ".pr-rd-author-submission-date span[data-datetime]").text.strip()
            except:
                date = ""   
            try:
                location = rev.find_element(By.CSS_SELECTOR, ".pr-rd-author-location span:last-child").text
            except:
                location = ""          
            try:
                rev.find_element(By.CSS_SELECTOR, ".pr-verified_buyer")
                verified = "Yes"
            except:
                verified = "No"
            try:
                recommend = rev.find_element(By.CSS_SELECTOR, ".pr-rd-bottomline").text
                recommend = "Yes" if "Yes" in recommend else "No"
            except:
                recommend = ""
            page.append({
            "rating":rating,
            "headline":headline,
            "text":text,
            "author":author,
            "date":date,
            "location":location,
            "verified_buyer":verified,
            "would_recommend":recommend
            })
        except Exception as e:
            print("ERROR IN EXTRACTING REVIEW")
    return page
all_reviews=[]
page_num=1
while True:
    page_reviews=extract_review()
    all_reviews.extend(page_reviews)
    try:
        next_link=driver.find_element(By.CSS_SELECTOR, "a.pr-rd-pagination-btn--next")
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});",next_link)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", next_link)
        time.sleep(3)
        WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".pr-review"))
        )
        reviews_section=driver.find_element(By.ID, "ratings_section")
        driver.execute_script("arguments[0].scrollIntoView(true);", reviews_section)
        time.sleep(1)
        page_num+=1
    except Exception as e:
        break
driver.quit()

In [8]:
df=pd.DataFrame(all_reviews)
df=df.drop_duplicates(subset=["headline", "author", "text"], keep="first")
df.to_csv("DELL_REVIEWS.csv",index=False)

In [9]:
df.head(10)

,rating,headline,text,author,date,location,verified_buyer,would_recommend
0,5,Martians are the best,This is my second Alienware I love my first on...,JD,2 days ago,From Wyoming,Yes,Yes
1,5,Terrific Gaming Laptop,This laptop is an excellent gaming machine. I'...,Zak,2 days ago,"From Chandler, Arizona",Yes,Yes
2,5,"Perfect size, Powerful performance, Convenient...",I wanted a machine that could do anything...Ga...,Ren,3 days ago,"From Monterey, California",Yes,Yes
3,5,Everything I wanted and more,This is the nicest thing I've ever owned. Powe...,Wamp,3 days ago,"From Pigeon Forge, TN",Yes,Yes
4,5,Nice Screen! Great Gaming Laptop!,"18""screen. Quick uploads and great framerates....",WARHAUS,3 days ago,From NC,Yes,Yes
5,5,very pleased,use it like a regular computer,joe,7 days ago,"From Buffalo,NY",Yes,Yes
6,5,Area51,5090 is an exceptional graphics system and it ...,Chip,12 days ago,From TX,Yes,Yes
7,5,Alien Awesomeness,The best laptop. I have had up to this date. S...,The Mad King,13 days ago,From Phoenix,Yes,Yes
8,5,Perfect for gaming,Compatible with all games.,Big,14 days ago,,Yes,Yes
9,5,Alienware Laptop - Out of this world!!,"Top notch quality, easy setup and, honestly, t...",MoneyMaker,17 days ago,From Michigan,Yes,Yes


2. ASUS REVIEW

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

In [11]:
max_reviews=18
options=Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches",["enable-automation"])

In [12]:
driver=webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",{
    "source":"Object.defineProperty(navigator.'webdriver',{get: () => undefined})"
}) 
url="https://shop.asus.com/us/90nr0lf1-m00720-rog-strix-scar-18-2025.html"
driver.get(url)
time.sleep(10)
try:
    driver.find_element(By.XPATH, "//button[contains(text(),'Accept all')]").click()
    time.sleep(2)
except:
    pass
for i in range(6):
    driver.execute_script(f"window.scrollTo(0, {400 * (i+1)});")
    time.sleep(0.5)
try:
    tab=driver.find_element(By.CSS_SELECTOR, "#tab-label-rating_info-accordion, #tab-label-rating_info")
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", tab)
    driver.execute_script("arguments[0].click();", tab)
    time.sleep(8)
except Exception as e:
    print("ERROR")

In [13]:
def extract_reviews():
    reviews = driver.execute_script("""
        let shadowHost = document.querySelector('div.rating-reviews');
        if (!shadowHost || !shadowHost.shadowRoot) {
            return [];
        }
        let shadow = shadowHost.shadowRoot;

        let reviews = [];
        let reviewSections = shadow.querySelectorAll('section[aria-label*="out of"]');

        reviewSections.forEach(function(sec) {
            try {
                let review = {};

                let aria = sec.getAttribute('aria-label') || '';
                let ratingMatch = aria.match(/Ratings\\s+(\\d)/);
                review.rating = ratingMatch ? ratingMatch[1] : 'N/A';

                let h3 = sec.querySelector('h3');
                review.headline = h3 ? h3.innerText.trim() : '';

                let textEl = sec.querySelector('div[id*="review-text"]');
                if (textEl) {
                    review.text = textEl.innerText.trim();
                } else {
                    let tabDiv = sec.querySelector('div[tabindex="-1"]');
                    review.text = tabDiv ? tabDiv.innerText.trim() : '';
                }

                let authorBtn = sec.querySelector('button[aria-label*="profile"]');
                if (authorBtn) {
                    let match = authorBtn.getAttribute('aria-label').match(/See\\s+(.+?)\\s+profile/);
                    review.author = match ? match[1] : 'Anonymous';
                } else {
                    let authorSpan = sec.querySelector('span[class*="1r4hv38"]');
                    review.author = authorSpan ? authorSpan.innerText.trim() : 'Anonymous';
                }

                let allSpans = sec.querySelectorAll('span');
                review.date = '';
                allSpans.forEach(function(span) {
                    let txt = span.innerText || '';
                    if (txt.match(/\\d+\\s*(day|month|year|week)s?\\s*ago/i) || txt.match(/a\\s+(day|month|year|week)\\s*ago/i)) {
                        review.date = txt.trim();
                    }
                });

                let locEl = sec.querySelector('div[class*="emkap-1"] span');
                review.location = locEl ? locEl.innerText.trim() : '';

                if (review.text || review.headline) {
                    reviews.push(review);
                }

            } catch(e) {
                console.log('Error parsing review: ' + e);
            }
        });

        return reviews;
    """)

    return reviews

In [14]:
def next_page():
    return driver.execute_script("""
        let shadowHost = document.querySelector('div.rating-reviews');
        if (!shadowHost || !shadowHost.shadowRoot) return false;
        let shadow = shadowHost.shadowRoot;
        let nextBtn = shadow.querySelector('svg[filter*="pagination_right"]');
        if (!nextBtn) {
            let btns = shadow.querySelectorAll('button, a, div[role="button"]');
            for (let btn of btns) {
                let svg = btn.querySelector('svg');
                if (svg && svg.innerHTML.includes('pagination_right')) {
                    nextBtn = btn;
                    break;
                }
            }
        }
        
        if (nextBtn) {
            let clickable = nextBtn.closest('button, a, div[role="button"]') || nextBtn.parentElement;
            if (clickable) {
                clickable.click();
                return true;
            }
        }
        return false;
    """)
    

In [15]:
all_reviews=[]
page_num=1
while True:
    page_reviews=extract_reviews()
    if page_reviews:
        all_reviews.extend(page_reviews)
        if len(all_reviews)>=max_reviews:
            all_reviews=all_reviews=all_reviews[:max_reviews]
            break
    time.sleep(1)
    if next_page():
        time.sleep(3)
        page_num+=1
    else:
        print("Completed Scrapping")
        break
driver.quit()

In [16]:
df=pd.DataFrame(all_reviews)
df=df.drop_duplicates(subset=["headline","text"],keep="first")
df.to_csv("asus1.csv",index=False)

In [17]:
df.head()

,author,date,headline,location,rating,text
0,UmerW,13 days ago,Expensive equipment with not much of value,"Virginia, USA",4,Just ok. I got the top of the line model in Oc...
1,Hibcity boy,17 days ago,The rgbs and back lights stands out from the rest,California,5,Its an awesome beasr of laptop battlefield 6 i...
2,Anonymous,28 days ago,FAST AND POWERFUL,Clearwater FL,5,"Excellent laptop, I recommend it 100%, it's a ..."
3,Anonymous,a month ago,Great laptop even better performance. Small is...,Idaho,5,Great laptop and even better performance. Upgr...
4,Judee,a month ago,Has issues,"Port Orange, Fl",3,Blue screen crashes. This is the second one I ...


In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

In [19]:
MAX_REVIEWS = 200
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

In [20]:
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
url = "https://www.bestbuy.com/site/reviews/asus-rog-strix-g18-18-2-5k-240hz-gaming-laptop-intel-core-ultra-9-hx-32gb-ram-nvidia-geforce-rtx-5080-2tb-ssd-eclipse-gray/6613958"
driver.get(url)
time.sleep(8)
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.l3skK7wJo_SxRa9G, li.review-item"))
    )
except Exception as e:
    print("Could not find reviews section")

In [21]:
def extract_reviews():
    page_reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, "li.l3skK7wJo_SxRa9G")
    if not review_elements:
        review_elements = driver.find_elements(By.CSS_SELECTOR, "li.review-item")
    for rev in review_elements:
        try:
            review = {}
            try:
                rating_text = rev.find_element(By.CSS_SELECTOR, ".review-rating .visually-hidden").text
                match = re.search(r'Rated\s+(\d+)', rating_text)
                review['rating'] = match.group(1) if match else "N/A"
            except:
                review['rating'] = "N/A"
            try:
                review['headline'] = rev.find_element(By.CSS_SELECTOR, ".review-title").text.strip()
            except:
                try:
                    review['headline'] = rev.find_element(By.CSS_SELECTOR, "h4.review-title").text.strip()
                except:
                    review['headline'] = ""
            try:
                review['text'] = rev.find_element(By.CSS_SELECTOR, ".ugc-review-body p").text.strip()
            except:
                try:
                    review['text'] = rev.find_element(By.CSS_SELECTOR, ".pre-white-space").text.strip()
                except:
                    review['text'] = ""
            try:
                review['author'] = rev.find_element(By.CSS_SELECTOR, ".ugc-author strong").text.strip()
            except:
                review['author'] = "Anonymous"
            try:
                review['date'] = rev.find_element(By.CSS_SELECTOR, "time").get_attribute("title")
            except:
                review['date'] = ""
            try:
                verified = rev.find_elements(By.CSS_SELECTOR, '[class*="verified"]')
                review['verified_buyer'] = "Yes" if verified else "No"
            except:
                review['verified_buyer'] = "No"
            try:
                rec_text = rev.find_element(By.CSS_SELECTOR, ".ugc-recommendation").text.lower()
                review['would_recommend'] = "No" if "not" in rec_text else "Yes"
            except:
                review['would_recommend'] = ""
            try:
                help_btn = rev.find_element(By.CSS_SELECTOR, ".helpfulness-button").text
                match = re.search(r'\((\d+)\)', help_btn)
                review['helpful_count'] = match.group(1) if match else "0"
            except:
                review['helpful_count'] = "0"
            try:
                imgs = rev.find_elements(By.CSS_SELECTOR, 'img[data-testid^="customer-image"]')
                review['has_images'] = "Yes" if imgs else "No"
                review['image_count'] = str(len(imgs))
            except:
                review['has_images'] = "No"
                review['image_count'] = "0"
            if review['text'] or review['headline']:
                page_reviews.append(review)
        except Exception as e:
            print("Error extracting review:", e)

    return page_reviews

In [22]:
all_reviews = []
page_num = 1
while True:
    page_reviews = extract_reviews()  
    if page_reviews:
        all_reviews.extend(page_reviews)
        if len(all_reviews) >= MAX_REVIEWS:
            all_reviews = all_reviews[:MAX_REVIEWS]
            break
    elif page_num == 1:
        break
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        next_page_num = page_num + 1
        next_link = driver.find_element(By.XPATH, f"//a[contains(@href, 'page={next_page_num}')]")   
        if next_link.is_displayed() and next_link.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_link)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_link)
            time.sleep(3)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.l3skK7wJo_SxRa9G, li.review-item"))
            )
            page_num += 1
        else:
            break
            
    except Exception as e:
        break
driver.quit()

In [23]:
df = pd.DataFrame(all_reviews)
df = df.drop_duplicates(subset=["headline", "author", "text"], keep="first")
df.to_csv("asus2.csv", index=False)

In [24]:
df.head()

,rating,headline,text,author,date,verified_buyer,would_recommend,helpful_count,has_images,image_count
0,5,Asus knocked it out of the park!,The Asus Strix G18 is a game-changing laptop t...,RobertH,"Apr 19, 2025 9:54 AM",Yes,Yes,3,No,0
1,4,Great with a caveat.,"Overall a great laptop. Great display, RTX 508...",Mallandyrr,"Aug 31, 2025 10:14 PM",Yes,Yes,0,No,0
2,4,A really solid performer but with some drawbacks,I started my journey with this laptop a little...,Lorenzo,"Sep 1, 2025 8:11 AM",Yes,Yes,4,No,0
3,5,Great Workstation for Design & Gaming,"Overall, from start to finish the set up and u...",JaysCreations,"May 18, 2025 12:31 PM",Yes,Yes,2,No,0
4,3,Good but had defective unit,"I purchased this to replace my ""Alienware - Ar...",VoIPTeK,"May 4, 2025 10:36 PM",Yes,No,5,No,0


In [25]:
import pandas as pd
import os
def combine(f1,f2,f3):
    try:
        df1=pd.read_csv(f1)
        df2=pd.read_csv(f2)
        combine=pd.concat([df1,df2],ignore_index=True)
        combine.to_csv(f3,mode='w',index=False)
    except FileNotFoundError as e:
        print("ERROR")
combine("asus1.csv","asus2.csv","ASUS_REVIEWS.csv")

3. Acer Review

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

In [27]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

In [28]:
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
url = "https://www.bestbuy.com/site/reviews/acer-predator-helios-18-ai-18-250hz-gaming-laptop-2560-x-1600-intel-core-ultra-9-nvidia-geforce-rtx-5080--32gb--1tb-abyssal-black/6621420"
driver.get(url)
time.sleep(8)
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".review-item, li.review-item"))
    )
except Exception as e:
    print("Error")

In [29]:
def extract_reviews():
    page_reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, "li.review-item")   
    for rev in review_elements:
        try:
            review = {}
            try:
                rating_el = rev.find_element(By.CSS_SELECTOR, ".c-stars-wrapper")
                rating_text = rev.find_element(By.CSS_SELECTOR, ".review-rating .visually-hidden").text
                import re
                match = re.search(r'Rated (\d+)', rating_text)
                review['rating'] = match.group(1) if match else "N/A"
            except:
                review['rating'] = "N/A"
            try:
                review['headline'] = rev.find_element(By.CSS_SELECTOR, ".review-title, h4.review-title").text.strip()
            except:
                review['headline'] = ""
            try:
                review['text'] = rev.find_element(By.CSS_SELECTOR, ".ugc-review-body p, .pre-white-space").text.strip()
            except:
                review['text'] = ""
            try:
                review['author'] = rev.find_element(By.CSS_SELECTOR, ".ugc-author strong").text.strip()
            except:
                review['author'] = "Anonymous"
            try:
                review['date'] = rev.find_element(By.CSS_SELECTOR, "time.submission-date").get_attribute("title")
            except:
                review['date'] = ""
            try:
                rev.find_element(By.CSS_SELECTOR, ".verified-purchaser-mv-wrapper, [class*='verified']")
                review['verified_buyer'] = "Yes"
            except:
                review['verified_buyer'] = "No"
            try:
                recommend_el = rev.find_element(By.CSS_SELECTOR, ".ugc-recommendation")
                recommend_text = recommend_el.text
                review['would_recommend'] = "No" if "not" in recommend_text.lower() else "Yes"
            except:
                review['would_recommend'] = ""
            try:
                helpful_btn = rev.find_element(By.CSS_SELECTOR, ".helpfulness-button")
                helpful_text = helpful_btn.text
                match = re.search(r'\((\d+)\)', helpful_text)
                review['helpful_count'] = match.group(1) if match else "0"
            except:
                review['helpful_count'] = "0"
            if review['text'] or review['headline']:
                page_reviews.append(review)
        except Exception as e:
            print("Error")
    return page_reviews

In [30]:
all_reviews = []
page_num = 1
while True:
    page_reviews = extract_reviews()
    if page_reviews:
        all_reviews.extend(page_reviews)
    elif page_num == 1:
        break
    try:
        next_btns = driver.find_elements(By.CSS_SELECTOR, ".page-pagination a[aria-label*='Next'], .footer-pagination a[aria-label*='Next']")
        if next_btns:
            next_btn = next_btns[0]
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_btn)
            time.sleep(3)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.review-item"))
            )
            page_num += 1
        else:
            break  
    except Exception as e:
        break
driver.quit()    

In [31]:
df = pd.DataFrame(all_reviews)
df = df.drop_duplicates(subset=["headline", "author", "text"], keep="first")
df.to_csv("acer1.csv", index=False)

In [32]:
df.head()

,rating,headline,text,author,date,verified_buyer,would_recommend,helpful_count
0,5,Fast and Powerful!,I've had it for about month and so I'm still a...,Tangledwolf,"Oct 26, 2025 3:21 PM",Yes,Yes,0
1,5,very satisfied,Screen mini led 10/10\nPerformance 10/10\...,GeraltofNvidia,"Aug 15, 2025 2:30 PM",Yes,Yes,2
2,5,Loptop review,Great laptop with lots of bells and whistles. ...,DavidS,"Oct 22, 2025 8:35 PM",Yes,Yes,1
3,4,Decent mid ranged laptop,Overall a good deal but I return mine as it wa...,Krisha,"Oct 20, 2025 7:42 AM",Yes,Yes,0
4,5,this thing is a beast!,i got this laptop because it had the best spec...,TerrenceB,"Aug 8, 2025 1:25 PM",Yes,Yes,2


In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

In [34]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

In [35]:
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
url = "https://www.bestbuy.com/site/reviews/acer-predator-helios-neo-16s-ai-gaming-laptop-16-oled-240hz-intel-core-ultra-9-nvidia-geforce-rtx-5070ti--32gb--1tb-obsidian-black/6619312"
driver.get(url)
time.sleep(8)
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.l3skK7wJo_SxRa9G, li.review-item"))
    )
except Exception as e:
    print("ERROR")

In [36]:
def extract_reviews():
    page_reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, "li.review-item")
    for rev in review_elements:
        try:
            review = {}
            try:
                rating_text = rev.find_element(By.CSS_SELECTOR, ".review-rating .visually-hidden").text
                match = re.search(r'Rated\s+(\d+)', rating_text)
                review['rating'] = match.group(1) if match else "N/A"
            except:
                review['rating'] = "N/A"
            try:
                review['headline'] = rev.find_element(By.CSS_SELECTOR, ".review-title").text.strip()
            except:
                try:
                    review['headline'] = rev.find_element(By.CSS_SELECTOR, "h4.review-title").text.strip()
                except:
                    review['headline'] = "" 
            try:
                review['text'] = rev.find_element(By.CSS_SELECTOR, ".ugc-review-body p").text.strip()
            except:
                try:
                    review['text'] = rev.find_element(By.CSS_SELECTOR, ".pre-white-space").text.strip()
                except:
                    review['text'] = ""
            try:
                review['author'] = rev.find_element(By.CSS_SELECTOR, ".ugc-author strong").text.strip()
            except:
                review['author'] = "Anonymous"
            try:
                review['date'] = rev.find_element(By.CSS_SELECTOR, "time").get_attribute("title")
            except:
                try:
                    review['date'] = rev.find_element(By.CSS_SELECTOR, "time.submission-date").get_attribute("title")
                except:
                    review['date'] = ""
            try:
                badges = rev.find_elements(By.CSS_SELECTOR, '[class*="verified"]')
                review['verified_buyer'] = "Yes" if badges else "No"
            except:
                review['verified_buyer'] = "No"
            try:
                recommend_el = rev.find_element(By.CSS_SELECTOR, ".ugc-recommendation")
                recommend_text = recommend_el.text
                review['would_recommend'] = "No" if "not" in recommend_text.lower() else "Yes"
            except:
                review['would_recommend'] = ""
            try:
                helpful_btn = rev.find_element(By.CSS_SELECTOR, ".helpfulness-button")
                helpful_text = helpful_btn.text
                match = re.search(r'\((\d+)\)', helpful_text)
                review['helpful_count'] = match.group(1) if match else "0"
            except:
                review['helpful_count'] = "0"
            try:
                images = rev.find_elements(By.CSS_SELECTOR, 'img[data-testid^="customer-image"]')
                review['has_images'] = "Yes" if images else "No"
                review['image_count'] = str(len(images))
            except:
                review['has_images'] = "No"
                review['image_count'] = "0"
            if review['text'] or review['headline']:
                page_reviews.append(review)
        except Exception as e:
            print(f"[ERROR] Failed to extract review: {e}")
    
    return page_reviews

In [37]:
all_reviews = []
page_num = 1
while True:
    page_reviews = extract_reviews()
    if page_reviews:
        all_reviews.extend(page_reviews)
    elif page_num == 1:
        break
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        next_page_num = page_num + 1
        next_link = driver.find_element(By.XPATH, f"//a[contains(@href, 'page={next_page_num}')]")
        if next_link.is_displayed() and next_link.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_link)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_link)
            time.sleep(3)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.review-item"))
            )
            page_num += 1
        else:
            break  
    except Exception as e:
        break
driver.quit()

In [38]:
df = pd.DataFrame(all_reviews)
df = df.drop_duplicates(subset=["headline", "author", "text"], keep="first")
df.to_csv("acer2.csv", index=False)

In [39]:
df.head()

,rating,headline,text,author,date,verified_buyer,would_recommend,helpful_count,has_images,image_count
0,3,Beast performer for the price — average elsewhere,"Good with some compromises, amazing for up to ...",Brian,"Jun 30, 2025 4:10 PM",Yes,No,17,No,0
1,5,BEST BANG FOR BUCK LAPTOP DURONG 1ST HALF OF 2025,Let me be clear: THIS THING SLAPS.\n\nTl;dr: i...,MrPMOSH,"Jul 9, 2025 8:27 PM",Yes,Yes,7,No,0
2,5,Fantastic Performance,"The performance on this system is impressive, ...",Steven,"Oct 12, 2025 9:54 AM",Yes,Yes,2,No,0
3,5,Thin little monster - Love this Laptop.,"Great Gaming Laptop - This thing flies, the CP...",OddOdds,"Sep 26, 2025 10:33 AM",Yes,Yes,2,No,0
4,5,“Acer Predator Helios Neo 16S AI,If you’re looking for a gaming laptop that lea...,SHIVAP,"Oct 8, 2025 10:22 PM",Yes,Yes,1,No,0


In [40]:
import pandas as pd
import os
def combine(f1,f2,f3):
    try:
        df1=pd.read_csv(f1)
        df2=pd.read_csv(f2)
        combine=pd.concat([df1,df2],ignore_index=True)
        combine.to_csv(f3,mode='w',index=False)
    except FileNotFoundError as e:
        print("ERROR")
combine("acer1.csv","acer2.csv","ACER_REVIEWS.csv")

4. Lenovo Review

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

In [42]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

In [43]:
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
url = "https://www.bestbuy.com/product/lenovo-legion-7i-16-2-5k-oled-gaming-laptop-intel-core-ultra-9-275hx-32gb-memory-nvidia-geforce-rtx-5070-1tb-ssd-glacier-white/JJGSH3Y5H9/sku/6617095#tabbed-customerreviews"
driver.get(url)
time.sleep(8)
try:
    see_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'See All Customer Reviews')]"))
    )
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", see_all_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", see_all_button)
    time.sleep(5)      
except Exception as e:
    print("ERROR")
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.review-item"))
    )
except Exception as e:
    print("ERROR")

In [44]:
def extract_reviews():
    page_reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, "li.review-item")
    for rev in review_elements:
        try:
            review = {}
            try:
                rating_text = rev.find_element(By.CSS_SELECTOR, ".review-rating .visually-hidden").text
                match = re.search(r'Rated\s+(\d+)', rating_text)
                review['rating'] = match.group(1) if match else "N/A"
            except:
                review['rating'] = "N/A"
            try:
                review['headline'] = rev.find_element(By.CSS_SELECTOR, ".review-title").text.strip()
            except:
                try:
                    review['headline'] = rev.find_element(By.CSS_SELECTOR, "h4.review-title").text.strip()
                except:
                    review['headline'] = ""
            try:
                review['text'] = rev.find_element(By.CSS_SELECTOR, ".ugc-review-body p").text.strip()
            except:
                try:
                    review['text'] = rev.find_element(By.CSS_SELECTOR, ".pre-white-space").text.strip()
                except:
                    review['text'] = ""
            try:
                review['author'] = rev.find_element(By.CSS_SELECTOR, ".ugc-author strong").text.strip()
            except:
                review['author'] = "Anonymous"
            try:
                review['date'] = rev.find_element(By.CSS_SELECTOR, "time").get_attribute("title")
            except:
                try:
                    review['date'] = rev.find_element(By.CSS_SELECTOR, "time.submission-date").get_attribute("title")
                except:
                    review['date'] = ""
            try:
                badges = rev.find_elements(By.CSS_SELECTOR, '[class*="verified"]')
                review['verified_buyer'] = "Yes" if badges else "No"
            except:
                review['verified_buyer'] = "No"
            try:
                recommend_el = rev.find_element(By.CSS_SELECTOR, ".ugc-recommendation")
                recommend_text = recommend_el.text
                review['would_recommend'] = "No" if "not" in recommend_text.lower() else "Yes"
            except:
                review['would_recommend'] = ""
            try:
                helpful_btn = rev.find_element(By.CSS_SELECTOR, ".helpfulness-button")
                helpful_text = helpful_btn.text
                match = re.search(r'\((\d+)\)', helpful_text)
                review['helpful_count'] = match.group(1) if match else "0"
            except:
                review['helpful_count'] = "0"
            try:
                images = rev.find_elements(By.CSS_SELECTOR, 'img[data-testid^="customer-image"]')
                review['has_images'] = "Yes" if images else "No"
                review['image_count'] = str(len(images))
            except:
                review['has_images'] = "No"
                review['image_count'] = "0"
            if review['text'] or review['headline']:
                    page_reviews.append(review)
        except Exception as e:
            print("Error")
    return page_reviews            

In [45]:
all_reviews = []
page_num = 1
while True:
    page_reviews = extract_reviews()
    if page_reviews:
        all_reviews.extend(page_reviews)
    elif page_num == 1:
        break 
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        next_page_num = page_num + 1
        next_link = driver.find_element(By.XPATH, f"//a[contains(@href, 'page={next_page_num}')]")   
        if next_link.is_displayed() and next_link.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_link)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_link)
            time.sleep(3)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.review-item"))
            )
            page_num += 1
        else:
            break
            
    except Exception as e:
        break
driver.quit()

In [46]:
df = pd.DataFrame(all_reviews)
df = df.drop_duplicates(subset=["headline", "author", "text"], keep="first")
df.to_csv("lenovo1.csv", index=False)

In [47]:
df.head()

,rating,headline,text,author,date,verified_buyer,would_recommend,helpful_count,has_images,image_count
0,5,Aprobada y recomendada,"I recommend it, it has great capacity to run h...",OctavioM,"Oct 17, 2025 1:29 PM",Yes,Yes,1,No,0
1,5,Upgrade Blows My Desktop Away,I purchased this laptop to replace my desktop ...,SHAE,"Sep 15, 2025 3:35 PM",Yes,Yes,3,No,0
2,5,Great Performance and Build Quality,This Lenovo Legion 7i is excellent. The build ...,EddyG,"Nov 14, 2025 8:03 PM",Yes,Yes,0,No,0
3,5,"Legion 7i: Speed, Style & Serious Performance",⭐️⭐️⭐️⭐️⭐️\nThe Legion 7i is a beast. Blazing ...,KennyW,"Oct 3, 2025 9:41 AM",Yes,Yes,1,No,0
4,5,"Lenovo Legion 7i Intel Core Ulta 9 275 HX, 32 ...","I'm a huge Lenovo fan, and I absolutely fell i...",JuandanielR,"Jul 18, 2025 2:46 PM",Yes,Yes,2,No,0


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

In [49]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

In [50]:
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
url = "https://www.bestbuy.com/product/lenovo-legion-pro-5i-16-2-5k-oled-gaming-laptop-intel-core-ultra-9-275hx-32gb-memory-nvidia-geforce-rtx-5070-1tb-ssd-eclipse-black/JJGSHXJRCS"
driver.get(url)
time.sleep(8)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.6);")
time.sleep(3)
try:
    see_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'See All Customer Reviews')]/ancestor::button"))
    )
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", see_all_button)
    time.sleep(1)
    see_all_button.click()
    time.sleep(5)
except Exception as e:
    print("ERROR")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.7);")
time.sleep(3)
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.review-item"))
    )
except Exception as e:
    print("ERROR")

In [51]:
def extract_reviews():
    page_reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, "li.review-item")
    for rev in review_elements:
        try:
            review = {}
            try:
                rating_text = rev.find_element(By.CSS_SELECTOR, ".visually-hidden").text
                match = re.search(r'Rated (\d+)', rating_text)
                review['rating'] = match.group(1) if match else "N/A"
            except:
                review['rating'] = "N/A"
            try:
                review['headline'] = rev.find_element(By.CSS_SELECTOR, "h4.review-title").text.strip()
            except:
                review['headline'] = ""
            try:
                review['text'] = rev.find_element(By.CSS_SELECTOR, "p.pre-white-space").text.strip()
            except:
                review['text'] = ""
            try:
                review['author'] = rev.find_element(By.CSS_SELECTOR, ".ugc-author strong").text.strip()
            except:
                review['author'] = "Anonymous"
            try:
                review['date'] = rev.find_element(By.CSS_SELECTOR, "time.submission-date").get_attribute("title")
            except:
                try:
                    review['date'] = rev.find_element(By.CSS_SELECTOR, "time").get_attribute("title")
                except:
                    review['date'] = ""
            try:
                verified = rev.find_elements(By.CSS_SELECTOR, '.verified-purchaser-sv-wrapper, .verified-purchaser-mv-wrapper')
                review['verified_buyer'] = "Yes" if verified else "No"
            except:
                review['verified_buyer'] = "No"
            try:
                recommend_el = rev.find_element(By.CSS_SELECTOR, ".ugc-recommendation")
                recommend_text = recommend_el.text
                review['would_recommend'] = "No" if "not" in recommend_text.lower() else "Yes"
            except:
                review['would_recommend'] = ""
            try:
                helpful_btn = rev.find_element(By.CSS_SELECTOR, ".helpfulness-button")
                helpful_text = helpful_btn.text
                match = re.search(r'\((\d+)\)', helpful_text)
                review['helpful_count'] = match.group(1) if match else "0"
            except:
                review['helpful_count'] = "0"
            try:
                images = rev.find_elements(By.CSS_SELECTOR, '.image-thumbnail')
                review['has_images'] = "Yes" if images else "No"
                review['image_count'] = str(len(images))
            except:
                review['has_images'] = "No"
            try:
                ownership = rev.find_element(By.CSS_SELECTOR, ".posted-date-ownership").text
                match = re.search(r'Owned for\s+(.+?)\s+when reviewed', ownership)
                review['ownership_duration'] = match.group(1) if match else ""
            except:
                review['ownership_duration'] = ""
            if review['text'] or review['headline']:
                page_reviews.append(review)
                
        except Exception as e:
            print("Error")
    return page_reviews

In [52]:
all_reviews = []
page_num = 1
while True:
    page_reviews = extract_reviews()
    if page_reviews:
        all_reviews.extend(page_reviews)
    elif page_num == 1:
        break
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        next_page_num = page_num + 1
        next_link = driver.find_element(By.CSS_SELECTOR, f'a[aria-label="Page {next_page_num}"]')
        if next_link.is_displayed() and next_link.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_link)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_link)
            time.sleep(4)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li.review-item"))
            )
            page_num += 1
        else:
            break
            
    except Exception as e:
        break
driver.quit()

In [53]:
df = pd.DataFrame(all_reviews)
df = df.drop_duplicates(subset=["headline", "author", "text"], keep="first")
df.to_csv("lenovo2.csv", index=False)

In [54]:
df.head()

,rating,headline,text,author,date,verified_buyer,would_recommend,helpful_count,has_images,image_count,ownership_duration
0,5,You cant go wrong with this laptoo,The best thing you can do if you want performa...,AddisonM,"Nov 16, 2025 10:00 AM",Yes,Yes,1,Yes,1,2 weeks
1,5,Spectacular laptop,This laptop works exceptionally! The temps are...,Gabriel,"Sep 5, 2025 7:43 AM",Yes,Yes,0,Yes,1,
2,5,The is a beast of a machine,What a power house … runs all of my cnc progra...,JaredG,"Aug 1, 2025 3:51 PM",Yes,Yes,0,Yes,2,1 month
3,5,a great buy,Very happy with the purchase process and the p...,NeiderU,"Nov 14, 2025 7:46 PM",Yes,Yes,0,Yes,1,
4,5,Lenovo,Best computer best graphics and sound as well ...,JasonR,"Oct 24, 2025 10:14 AM",Yes,Yes,0,Yes,1,3 weeks


In [55]:
import pandas as pd
import os
def combine(f1,f2,f3):
    try:
        df1=pd.read_csv(f1)
        df2=pd.read_csv(f2)
        combine=pd.concat([df1,df2],ignore_index=True)
        combine.to_csv(f3,mode='w',index=False)
    except FileNotFoundError as e:
        print("ERROR")
combine("lenovo1.csv","lenovo2.csv","LENOVO_REVIEWS.csv")